In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [3]:
from sklearn import preprocessing

In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras.models import model_from_json

In [6]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [7]:
data = pd.read_csv("spam.csv",encoding='latin-1')

In [8]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [45]:
y = list(data['v1'])
x = list(data['v2'])

In [46]:
le = preprocessing.LabelEncoder()

In [47]:
le.fit(y)
le.classes_

array(['ham', 'spam'], dtype='<U4')

In [48]:
def encode(le,labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

In [49]:
def decode(le,one_hot):
    dec = np.argmax(one_hot,axis=1)
    return le.inverse_transform(dec)

In [50]:
test = encode(le,['ham','spam','ham','ham'])
test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [53]:
untest = decode(le,y_test)
untest

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [52]:
x_enc = x
y_enc = encode(le,y)
x_train = np.asarray(x_enc[:3850])
y_train = np.asarray(y_enc[:3850])
x_test = np.asarray(x_enc[3850:])
y_test = np.asarray(y_enc[3850:])

In [27]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model,load_model
import keras.backend as K

In [18]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default" ,as_dict = True)["default"]

In [19]:
input_text = Input(shape=(1,) ,dtype=tf.string)
embedding = Lambda(ELMoEmbedding,output_shape=(1024,))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs =[input_text],outputs =pred)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [31]:
tester = "U dun say so early hor... U c already then say..."

In [32]:
testing_list=[]
testing_list.append(tester)

In [33]:
testing_list.append('')
pred_test = np.asarray(testing_list)

In [29]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(x_train,y_train,epochs=2,batch_size = 32)
    model.save_weights('./elmo_model.h5')

Epoch 1/2
3850/3850 [==============================] - 561s 146ms/step - loss: 0.0790 - accuracy: 0.9732
Epoch 2/2
3850/3850 [==============================] - 517s 134ms/step - loss: 0.0307 - accuracy: 0.9891


In [34]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo_model.h5')
    #predicts = model.predict(x_test,batch_size=32)
    predicts = model.predict(pred_test)

In [42]:
decode(le,y_test)

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [54]:
y_test = decode(le,y_test)
y_preds = decode(le,predicts)
print(y_preds[0])

ham


In [24]:
from sklearn import metrics

In [25]:
metrics.confusion_matrix(y_test,y_preds)

array([[1476,    9],
       [  19,  218]], dtype=int64)

In [26]:
print(metrics.classification_report(y_test,y_preds))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1485
        spam       0.96      0.92      0.94       237

    accuracy                           0.98      1722
   macro avg       0.97      0.96      0.97      1722
weighted avg       0.98      0.98      0.98      1722

